# `MySQL & Excel Table converted to 3 dataframes`

# `Helping a Viewer`

# <font color=red>Mr Fugu Data Science</font>

# (◕‿◕✿)

**`Help Support The Channel: Patreon @mrfugudatasci`**

**`Or Consider Buy Me A Coffee @mrfugudatasci`**

In [1]:
import pandas as pd  # Create Dataframe

import os,re,datetime

In [9]:
# Find file from your directory:

def os_any_dir_search(file):
    u=[]
    for p,n,f in os.walk(os.getcwd()):
        
        for a in f:
            a = str(a)
            if a.endswith(file): # can be (.csv) or a file like I did and search 
#                 print(a)
#                 print(p)
#                 t=pd.read_json(p+'/'+file)
                u.append(p+'/')
    return u


# os_any_dir_search('Master.xlsx')

In [10]:
# call the file I want with my working directory:

our_dta=pd.ExcelFile('Master.xlsx') # this is slow!!!


**`Since, this file was NOT used as separate tabs/sheets it makes parsing a little challenging.`**
+ When you use sheets we can do: our_dta.sheet_names
    + then we can go into pandas and do `pd.read_excel(our_dta,'Sheet to enter',and any headers here)`
    
But, now we have to create 3 tables based on parsing using Regex or string matching!
    + You need to split baed on these values then.

In [11]:
our_dta.sheet_names


['Master']

In [19]:
three_tbls_n_one=pd.read_excel(our_dta)

three_tbls_n_one

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3
0,Table Name: Country Master,NaN,NaN,NaN
1,Last Update Date: 20190421,NaN,NaN,NaN
2,Country,Group,NaN,NaN
3,Bangladesh,Asia,NaN,NaN
4,Bhutan,Asia,NaN,NaN
...,...,...,...,...
1207,SUPA101,Super Soft - Product Sample,0.3,0
1208,SUPA102,Super Soft - 250ml,2.5,4.5
1209,SUPA103,Super Soft - 500ml,3.5,6.99
1210,SUPA104,Super Soft - 1 Litre,5,9.99


# `Pay Attention to first 2 entries:`

+ `Entry 1:` ['Table Name: Country Master', nan, nan, nan]
    + we will save the table name of `Country` if we want 
+ `Entry 2:` ['Last Update Date: 20190421', nan, nan, nan]  
    + last update will need to be stored and made into a separate column. 
    + We will need to also make it the same length as the dataframe as well.

In [20]:
three_tbls_n_one.values

array([['Table Name: Country Master', nan, nan, nan],
       ['Last Update Date: 20190421', nan, nan, nan],
       ['Country', 'Group', nan, nan],
       ...,
       ['SUPA103', 'Super Soft - 500ml', 3.5, 6.99],
       ['SUPA104', 'Super Soft - 1 Litre', 5, 9.99],
       ['SUPA105', 'Super Soft Bulk - 2 Litres', 8, 14.5]], dtype=object)

# `We need to separate the excel file into 3 tables/DF's`

+ Finding the position index value where we have string 'Table Name' will be our best bet.
    + This is the division point where the table starts (we have 3 tables) thus three index positions
    
(find the index position) so we can then separate the DF by these division points by rows.

In [25]:
row_idx=[]
for i in range(len(three_tbls_n_one.values)):

    if type(three_tbls_n_one.values[i][0] )!=str: # since we have nan values
        pass
    elif 'Table Name:' in three_tbls_n_one.values[i][0]: # sub str comparison

        row_idx.append(i)

# where we split rows in DF:
row_idx

[0, 195, 1197]

# `Table 01:`

+ We will use the row numbers [0,194]
    + Also, the first 2 rows are the table name and date updated
+ `Date updated:` will be used for an added column later of same length as DF

In [27]:
# indexing proper rows: and removing first 2 rows
country_01=three_tbls_n_one.iloc[range(row_idx[0],row_idx[1]-1),:-2].values[2:]

df=pd.DataFrame(country_01)

# extract the first row with the new column names
df=df.rename(columns=df.iloc[0])

# drop the first row because it was the column names
df.drop(df.index[0])

,Country,Group
1,Bangladesh,Asia
2,Bhutan,Asia
3,Brunei,Asia
4,Cambodia,Asia
5,China,Asia
...,...,...
187,Samoa,Australia and Oceania
188,Solomon Islands,Australia and Oceania
189,Tonga,Australia and Oceania
190,Tuvalu,Australia and Oceania


# `Table 01: dates`

+ We will take the first date based on the position of indexed row:
    + The 3 indexed rows will each correspond to the division point of each table.
        + The first 2 rows of each table relate to the:
            + Table name
            + date updated
knowing this we are able use extract the dates, by indexed rows.

In [34]:
# Take first row which is 2 strings: first updated and date sep. by colon
# from row 0:194, remove first 2 rows and  extract the date
t=three_tbls_n_one.iloc[range(row_idx[0],row_idx[1]-1),:-2].values[1][0]

# take the string, split on :, this is because the date information we need then 
# multiply by length of DF to create a pandas sereis/df to concat later

last_update_country01=pd.DataFrame([t.split(':')[1].strip()]*len(df),
                                   columns=[t.split(':')[0]])

df_1=pd.concat([df,last_update_country01],axis=1)

df_1=df_1.drop(df_1.index[0]) # drop first row because its our column names
df_1

# convert string date to datetime format:
df_1['Last Update Date']=pd.to_datetime(df_1['Last Update Date'],
                    infer_datetime_format=True)

df_1


,Country,Group,Last Update Date
1,Bangladesh,Asia,2019-04-21
2,Bhutan,Asia,2019-04-21
3,Brunei,Asia,2019-04-21
4,Cambodia,Asia,2019-04-21
5,China,Asia,2019-04-21
...,...,...,...
187,Samoa,Australia and Oceania,2019-04-21
188,Solomon Islands,Australia and Oceania,2019-04-21
189,Tonga,Australia and Oceania,2019-04-21
190,Tuvalu,Australia and Oceania,2019-04-21


# `Table 02:`

In [29]:
# rows: 195:1196, exclude last two columns and remove first 2 rows
country_02=three_tbls_n_one.iloc[range(row_idx[1],row_idx[2]-1),:-2].values[2:]

df_2=pd.DataFrame(country_02)

# rename columns from first row
df_2=df_2.rename(columns=df_2.iloc[0])

# then drop first row
df_2.drop(df_2.index[0])

#  we need to now: pad id with 2 zeros need length 7
# also, add date column

,custId,custName
1,23262,Candice Levy
2,23263,Xerxes Smith
3,23264,Levi Douglas
4,23265,Uriel Benton
5,23266,Celeste Pugh
...,...,...
994,24255,Ethan Day
995,24256,Quail Knox
996,24257,Noelle Sargent
997,24258,Kuame Wallace


In [30]:
# retreive dates by 2nd table indexing of rows: 195:1196, removing first 2 rows
# and extracting date info
t_=three_tbls_n_one.iloc[range(row_idx[1],row_idx[2]-1),:-2].values[1][0]


# create the date column by stripping the string with its info, then multiply
# to match DF len and create new DF/series to concat
last_update_country02=pd.DataFrame([t_.split(':')[1].strip()]*len(df_2),
                                   columns=[t_.split(':')[0]])

df_2=pd.concat([df_2,last_update_country02],axis=1)

# dropping row info with table/date info that we don't need anymore
df_2=df_2.drop(df_2.index[0])
df_2

# convert string date to date/time
df_2['Last Update Date']=pd.to_datetime(df_2['Last Update Date'],
                    infer_datetime_format=True)
df_2

,custId,custName,Last Update Date
1,23262,Candice Levy,2019-04-21
2,23263,Xerxes Smith,2019-04-21
3,23264,Levi Douglas,2019-04-21
4,23265,Uriel Benton,2019-04-21
5,23266,Celeste Pugh,2019-04-21
...,...,...,...
994,24255,Ethan Day,2019-04-21
995,24256,Quail Knox,2019-04-21
996,24257,Noelle Sargent,2019-04-21
997,24258,Kuame Wallace,2019-04-21


In [13]:
# Change ID to Len 7, AKA padding with trailing zeroes: (CAN'T do leading zeroes)
# with integer btw, only strings

str(df_2['custId'].values[1]).ljust(7, '0')

chng_id_len7=[]
for i in range(len(df_2['custId'].values)):
    
# first convert int to str for formatting (padding)
# then pad to 7 digits using trailing zeroes

    chng_id_len7.append(int(str(df_2['custId'].values[i]).ljust(7, '0')))

df_2['custId']=chng_id_len7 # update column from int to date
df_2

,custId,custName,Last Update Date
1,2326200,Candice Levy,2019-04-21
2,2326300,Xerxes Smith,2019-04-21
3,2326400,Levi Douglas,2019-04-21
4,2326500,Uriel Benton,2019-04-21
5,2326600,Celeste Pugh,2019-04-21
...,...,...,...
994,2425500,Ethan Day,2019-04-21
995,2425600,Quail Knox,2019-04-21
996,2425700,Noelle Sargent,2019-04-21
997,2425800,Kuame Wallace,2019-04-21


# `Table 03:`

In [31]:
# take last 1197 to end range of rows, exclude last two columns and remove first 2 rows
country_03=three_tbls_n_one.iloc[row_idx[2]:,:-2].values[2:]

df_3=pd.DataFrame(country_03)

# rename columns from the first row
df_3=df_3.rename(columns=df_3.iloc[0])

# now drop the useless row
df_3.drop(df_3.index[0])

# next, add date column

,productId,productName
1,PURA100,Pure Soft Detergent - 100ml
2,PURA200,Pure Soft Detergent - 200ml
3,PURA250,Pure Soft Detergent - 250ml
4,PURA500,Pure Soft Detergent - 500ml
5,DETA100,Detafast Stain Remover - 100ml
6,DETA200,Detafast Stain Remover - 200ml
7,DETA800,Detafast Stain Remover - 800ml
8,SUPA101,Super Soft - Product Sample
9,SUPA102,Super Soft - 250ml
10,SUPA103,Super Soft - 500ml


In [32]:
# getting the row with our date info to strip string
t_3=three_tbls_n_one.iloc[row_idx[2]:,:-2].values[1][0]

# create new DF using date str then multiply to DF len and concat
last_update_country03=pd.DataFrame([t_3.split(':')[1].strip()]*len(df_3),
                                   columns=[t_3.split(':')[0]])


df_3=pd.concat([df_3,last_update_country03],axis=1)

# drop first row 
df_3=df_3.drop(df_3.index[0])

# create date column
df_3['Last Update Date']=pd.to_datetime(df_3['Last Update Date'],
                    infer_datetime_format=True)
df_3

,productId,productName,Last Update Date
1,PURA100,Pure Soft Detergent - 100ml,2019-04-21
2,PURA200,Pure Soft Detergent - 200ml,2019-04-21
3,PURA250,Pure Soft Detergent - 250ml,2019-04-21
4,PURA500,Pure Soft Detergent - 500ml,2019-04-21
5,DETA100,Detafast Stain Remover - 100ml,2019-04-21
6,DETA200,Detafast Stain Remover - 200ml,2019-04-21
7,DETA800,Detafast Stain Remover - 800ml,2019-04-21
8,SUPA101,Super Soft - Product Sample,2019-04-21
9,SUPA102,Super Soft - 250ml,2019-04-21
10,SUPA103,Super Soft - 500ml,2019-04-21


# `Last NOTE:`

+ if you wanted to save the data like the original format, you would need to create a header with the first 2 rows containing same information.
+ Or consider putting the first two rows of dataframe as table name and update info. then save as a csv or xlsx file.

# <font color=red>Like</font>, Share &

# <font color=red>SUB</font>scribe

**`Help Support the Channel: Patreon @mrfugudatasci`**

**`Or Buy Me A Coffee: @mrfugudatasci`**

# `Citations & Help:`


https://stackoverflow.com/questions/26521266/using-pandas-to-pd-read-excel-for-multiple-worksheets-of-the-same-workbook

https://stackoverflow.com/questions/26147180/convert-row-to-column-header-for-pandas-dataframe

https://stackoverflow.com/questions/42887663/reading-multiple-tabs-from-excel-in-different-dataframes

https://stackoverflow.com/questions/22436028/cant-connect-to-local-mysql-server-through-socket-tmp-mysql-sock-2

https://stackoverflow.com/questions/4891676/removing-duplicates-from-a-sql-query-not-just-use-distinct